In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import json

# Add src directory to path for modular imports
script_dir = os.path.dirname(os.path.abspath(''))
src_dir = os.path.abspath(os.path.join(script_dir, "src"))
if src_dir not in sys.path:
    sys.path.append(src_dir)

print("🔬 Synthetic Data Explorer & Generator")
print("=====================================")
print("This notebook demonstrates the complete synthetic data pipeline")
print("for magnetic distortion classification.\n")

# Try to use new modular imports
try:
    from mcap_utils import read_synthetic_sensor_data, plot_synthetic_sensor_data, extract_imu_windows
    from synthetic import SyntheticDataGenerator
    print("✅ Modular architecture loaded successfully")
    modular_available = True
    
except ImportError as e:
    print(f"⚠️ Modular imports failed: {e}")
    print("Trying legacy imports...")
    try:
        import mcap_utilities
        from synthetic import SyntheticDataGenerator  # This should still work
        read_synthetic_sensor_data = mcap_utilities.read_synthetic_sensor_data
        plot_synthetic_sensor_data = mcap_utilities.plot_synthetic_sensor_data
        extract_imu_windows = mcap_utilities.extract_imu_windows
        print("✅ Legacy imports successful")
        modular_available = False
    except ImportError as e2:
        print(f"❌ All imports failed: {e2}")
        print("Please check your environment setup")
        modular_available = False

if modular_available:
    # Demo 1: Generate Synthetic Data Live
    print("\n🧪 Demo 1: Live Synthetic Data Generation")
    print("-" * 45)
    
    try:
        generator = SyntheticDataGenerator()
        print("✅ SyntheticDataGenerator initialized")
        
        # Create a simple demonstration plan
        demo_plan = {
            "initialization": {
                "pose": {
                    "origin": {"lat": 38.446, "lng": -122.687, "height": 0.0},
                    "position": {"x": 0.0, "y": 0.0, "z": 0.0},
                    "rotation": {"w": 1.0, "x": 0.0, "y": 0.0, "z": 0.0}
                },
                "start_time_ns": 0,
                "sample_rate": 50,  # Lower sample rate for demo
                "mag": {"calibration": {"bias": {"x": 0.0, "y": 0.0, "z": 0.0}, "matrix": [[0.00053, 0.0, 0.0], [0.0, 0.00053, 0.0], [0.0, 0.0, 0.00053]]}},
                "acc": {"calibration": {"bias": {"x": 0.0, "y": 0.0, "z": 0.0}, "matrix": [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]}},
                "gyro": {"calibration": {"bias": {"x": 0.0, "y": 0.0, "z": 0.0}, "matrix": [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]}}
            },
            "segments": [
                {
                    "name": "demo_none",
                    "duration_s": 10.0,
                    "rotation_rpy_degrees": {"roll": 30.0, "pitch": 0.0, "yaw": 0.0},
                    "magnetic_distortion": 0.0,
                    "mag_distortion": {"level": "none"}
                },
                {
                    "name": "demo_high",
                    "duration_s": 10.0,
                    "rotation_rpy_degrees": {"roll": 0.0, "pitch": 30.0, "yaw": 0.0},
                    "magnetic_distortion": 2.5,
                    "mag_distortion": {"level": "high"}
                }
            ]
        }
        
        # Generate demo data
        demo_file = "demo_synthetic_data.mcap"
        demo_labels = "demo_synthetic_data.labels.json"
        
        print("? Generating 20 seconds of demo data...")
        generator.generate(plan_data=demo_plan, output_file=demo_file, verbose=False)
        generator.generate_labels(demo_plan, demo_labels)
        
        if os.path.exists(demo_file):
            file_size = os.path.getsize(demo_file)
            print(f"✅ Generated {demo_file} ({file_size:,} bytes)")
            
            # Read and analyze the generated data
            print("\n📊 Analyzing Generated Data...")
            data = read_synthetic_sensor_data(demo_file)
            
            print(f"📋 Available channels ({len(data)}):")
            for channel, channel_data in data.items():
                if len(channel_data) > 0:
                    timestamps = [item[0] for item in channel_data[:5]]  # First 5 timestamps
                    print(f"  📈 {channel}: {len(channel_data)} samples")
                    if len(channel_data) > 0:
                        sample_timestamp, sample_values = channel_data[0]
                        print(f"      Sample: t={sample_timestamp/1e9:.3f}s, values={sample_values}")
            
            # Analyze labels
            print(f"\n🏷️  Analyzing Labels...")
            with open(demo_labels, 'r') as f:
                labels_data = json.load(f)
                events = labels_data.get('events', [])
                print(f"📋 Found {len(events)} labeled events:")
                for i, event in enumerate(events):
                    name = event.get('metadata', {}).get('mag_distortion', 'unknown')
                    start_time = event['startTime']['sec']
                    end_time = event['endTime']['sec']
                    duration = end_time - start_time
                    print(f"  {i+1}. {name} distortion: {start_time}s - {end_time}s ({duration}s)")
        
        else:
            print("❌ Failed to generate demo data")
            
    except Exception as e:
        print(f"❌ Error in synthetic data generation: {e}")
    
    # Demo 2: Data Analysis and Visualization
    print(f"\n📊 Demo 2: Data Analysis & Visualization")
    print("-" * 45)
    
    if 'demo_file' in locals() and os.path.exists(demo_file):
        try:
            # Extract windows for ML
            print("🔄 Extracting windows for machine learning...")
            windows = extract_imu_windows(demo_file, window_size_ns=2e9, channels=['mag_truth', 'acc_truth', 'gyro_truth'])
            
            if len(windows) > 0:
                print(f"✅ Extracted {len(windows)} windows")
                print(f"   Window shape: {windows[0].shape}")
                print(f"   Data type: {windows[0].dtype}")
                
                # Show some statistics
                print(f"\n📈 Window Statistics:")
                all_data = np.concatenate(windows, axis=0)
                print(f"   Combined shape: {all_data.shape}")
                print(f"   Data range: [{np.min(all_data):.3f}, {np.max(all_data):.3f}]")
                print(f"   Mean: {np.mean(all_data):.3f}")
                print(f"   Std: {np.std(all_data):.3f}")
            else:
                print("⚠️ No windows extracted")
                
        except Exception as e:
            print(f"⚠️ Could not extract windows: {e}")
    
    # Demo 3: Integration with ML Pipeline
    print(f"\n🤖 Demo 3: Machine Learning Integration")
    print("-" * 45)
    
    try:
        from transformers import AutoFeatureExtractor, ASTForAudioClassification
        
        # Load a pretrained model for demonstration
        print("🔄 Loading pretrained Audio Spectrogram Transformer...")
        feature_extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
        model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
        
        print("✅ Loaded pretrained AST model")
        print(f"   📊 Model classes: {model.config.num_labels}")
        print(f"   🏷️  Sample classes: {list(model.config.id2label.values())[:5]}...")
        
        # Show how to create spectrograms
        if 'demo_file' in locals() and os.path.exists(demo_file):
            try:
                from mcap_utils import spectrogram_from_timeseries
                spectrogram_file = "demo_synthetic_data.spectrogram.mcap"
                
                print(f"🔄 Creating spectrogram from synthetic data...")
                spectrogram_from_timeseries(demo_file, spectrogram_file, feature_extractor=feature_extractor)
                
                if os.path.exists(spectrogram_file):
                    spec_size = os.path.getsize(spectrogram_file)
                    print(f"✅ Created spectrogram: {spectrogram_file} ({spec_size:,} bytes)")
                    print("   🎯 Ready for audio classification training!")
                
            except Exception as e:
                print(f"⚠️ Could not create spectrogram: {e}")
        
    except ImportError as e:
        print(f"⚠️ Transformers not available: {e}")
        print("   Install with: pip install transformers torch")
    
    # Demo 4: Multiple Distortion Levels
    print(f"\n🎚️  Demo 4: Multiple Distortion Levels")
    print("-" * 45)
    
    distortion_levels = ["none", "low", "high"]
    distortion_values = [0.0, 1.0, 2.5]
    
    print("🔄 Generating data for all distortion levels...")
    
    for level, value in zip(distortion_levels, distortion_values):
        try:
            quick_plan = {
                "initialization": {
                    "pose": {"origin": {"lat": 38.446, "lng": -122.687, "height": 0.0}, "position": {"x": 0.0, "y": 0.0, "z": 0.0}, "rotation": {"w": 1.0, "x": 0.0, "y": 0.0, "z": 0.0}},
                    "start_time_ns": 0,
                    "sample_rate": 30,
                    "mag": {"calibration": {"bias": {"x": 0.0, "y": 0.0, "z": 0.0}, "matrix": [[0.00053, 0.0, 0.0], [0.0, 0.00053, 0.0], [0.0, 0.0, 0.00053]]}},
                    "acc": {"calibration": {"bias": {"x": 0.0, "y": 0.0, "z": 0.0}, "matrix": [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]}},
                    "gyro": {"calibration": {"bias": {"x": 0.0, "y": 0.0, "z": 0.0}, "matrix": [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]}}
                },
                "segments": [{
                    "name": f"quick_{level}",
                    "duration_s": 5.0,
                    "rotation_rpy_degrees": {"roll": 45.0, "pitch": 0.0, "yaw": 0.0},
                    "magnetic_distortion": value,
                    "mag_distortion": {"level": level}
                }]
            }
            
            quick_file = f"quick_{level}.mcap"
            generator.generate(plan_data=quick_plan, output_file=quick_file, verbose=False)
            
            if os.path.exists(quick_file):
                # Read magnetometer data to see the effect
                data = read_synthetic_sensor_data(quick_file, channels=["mag_truth", "mag_raw"])
                
                if "mag_truth" in data and len(data["mag_truth"]) > 0:
                    mag_values = [item[1] for item in data["mag_truth"][:100]]  # First 100 samples
                    mag_magnitudes = [np.sqrt(sum(v**2 for v in vals)) for vals in mag_values]
                    
                    avg_magnitude = np.mean(mag_magnitudes)
                    print(f"   🧲 {level:>4} distortion (value={value:>3.1f}): avg magnitude = {avg_magnitude:.3f}")
                
        except Exception as e:
            print(f"   ❌ Error with {level} level: {e}")

else:
    print("❌ Cannot run demos - required modules not available")
    print("Please check your environment setup")

print(f"\n🎯 Next Steps:")
print("="*50)
print("1. 🏋️  Use fine_tune.ipynb to train a model with synthetic data")
print("2. 🎯 Use classify.ipynb to test classification on new synthetic data")
print("3. 🔧 Modify motion plans and distortion levels for custom scenarios")
print("4. 📊 Analyze the synthetic data to understand sensor behavior")
print("5. 🚀 Deploy trained models for real-world magnetic distortion detection")

print(f"\n💡 Key Benefits of Synthetic Data:")
print("- 🎛️  Full control over distortion levels and motion patterns")
print("- 📈 Unlimited training data generation")
print("- 🧪 Perfect ground truth labels")
print("- 🔄 Reproducible experiments")
print("- 💰 No need for expensive real sensor data collection")

print(f"\n🎉 Synthetic data exploration completed!")

0.17

# Dataset Exploration and Capabilities

This notebook demonstrates the capabilities of the time-series-classifier project's new modular architecture.

## Features Demonstrated:

### 1. **Synthetic Data Generation**
- Uses the new `SyntheticDataGenerator` class
- Reads and analyzes synthetic sensor data (magnetometer, accelerometer, gyroscope)
- Shows how synthetic data integrates with the ML pipeline

### 2. **Modular MCAP Utilities**
- Demonstrates the new `mcap_utils` module structure
- Shows window extraction for machine learning applications
- Illustrates data reading and analysis capabilities

### 3. **HuggingFace Integration**
- Shows how the system integrates with transformers and datasets
- Demonstrates the Audio Spectrogram Transformer (AST) model
- Illustrates the complete ML pipeline

### 4. **Improved Architecture**
- Showcases the new modular imports
- Demonstrates fallback capabilities for different environments
- Shows integration between synthetic data and ML models

## Key Improvements:
- **Modular Design**: Clean separation of concerns across modules
- **Better Error Handling**: Graceful fallbacks and clear error messages
- **Synthetic Data Support**: First-class support for synthetic sensor data
- **ML Pipeline Integration**: Seamless integration with HuggingFace ecosystem